In [2]:
import requests
import pandas as pd

base_url = "https://api.jolpi.ca/ergast/f1/2025/results/"
limit = 30  # L'API retourne 30 résultats par défaut
offset = 0
all_results = []

while True:
    url = f"{base_url}?limit={limit}&offset={offset}"
    response = requests.get(url)

    if response.status_code != 200:
        raise Exception("Erreur lors de la récupération des données.")

    data = response.json()
    races = data['MRData']['RaceTable']['Races']

    if not races:
        break  # Plus de résultats

    for race in races:
        for result in race['Results']:
            all_results.append({
                'Round': int(race['round']),
                'Race': race['raceName'],
                'Date': race['date'],
                'Driver': result['Driver']['familyName'],
                'Constructor': result['Constructor']['name'],
                'Grid': int(result['grid']),
                'Position': int(result['position']),
                'Status': result['status'],
                'Points': float(result['points'])
            })

    offset += limit  # Passer à la prochaine page

# Créer le DataFrame
df = pd.DataFrame(all_results)

# Trier les résultats pour plus de lisibilité
df.sort_values(by=['Round', 'Position'], inplace=True)

# Affichage dans un environnement Jupyter ou impression
try:
    display(df)
except:
    print(df.head())

# Sauvegarde optionnelle
df.to_csv("resultats_f1_2025_complets.csv", index=False)

,Round,Race,Date,Driver,Constructor,Grid,Position,Status,Points
0,1,Australian Grand Prix,2025-03-16,Norris,McLaren,1,1,Finished,25.0
1,1,Australian Grand Prix,2025-03-16,Verstappen,Red Bull,3,2,Finished,18.0
2,1,Australian Grand Prix,2025-03-16,Russell,Mercedes,4,3,Finished,15.0
3,1,Australian Grand Prix,2025-03-16,Antonelli,Mercedes,16,4,Finished,12.0
4,1,Australian Grand Prix,2025-03-16,Albon,Williams,6,5,Finished,10.0
...,...,...,...,...,...,...,...,...,...
135,7,Emilia Romagna Grand Prix,2025-05-18,Colapinto,Alpine F1 Team,16,16,Finished,0.0
136,7,Emilia Romagna Grand Prix,2025-05-18,Bearman,Haas F1 Team,19,17,Finished,0.0
137,7,Emilia Romagna Grand Prix,2025-05-18,Bortoleto,Sauber,14,18,Finished,0.0
138,7,Emilia Romagna Grand Prix,2025-05-18,Antonelli,Mercedes,13,19,Retired,0.0


In [3]:
# On ne garde que les arrivées terminées
df = df[df['Status'] == 'Finished']

# Trier par numéro de manche
df.sort_values(by='Round', inplace=True)

In [4]:
# Total des points par pilote
points_pilotes = df.groupby('Driver')['Points'].sum().sort_values(ascending=False)

# Nombre de victoires par pilote (Position == 1)
victoires_pilotes = df[df['Position'] == 1].groupby('Driver').size().sort_values(ascending=False)

# Fusion des deux classements
classement_pilotes = pd.DataFrame({
    'Points': points_pilotes,
    'Victoires': victoires_pilotes
}).fillna(0).astype({'Victoires': int}).sort_values(by='Points', ascending=False)

print("🏆 Classement Pilotes :")
print(classement_pilotes.head(10))

🏆 Classement Pilotes :
            Points  Victoires
Driver                       
Piastri      132.0          4
Norris       124.0          1
Verstappen   118.0          2
Russell       89.0          0
Leclerc       57.0          0
Antonelli     44.0          0
Albon         40.0          0
Hamilton      39.0          0
Ocon          14.0          0
Sainz         11.0          0


In [5]:
points_constructeurs = df.groupby('Constructor')['Points'].sum().sort_values(ascending=False)

print("\n🏢 Classement Constructeurs :")
print(points_constructeurs.head(10))


🏢 Classement Constructeurs :
Constructor
McLaren           256.0
Mercedes          133.0
Red Bull          122.0
Ferrari            96.0
Williams           51.0
Haas F1 Team       20.0
Aston Martin       10.0
RB F1 Team          7.0
Alpine F1 Team      6.0
Sauber              6.0
Name: Points, dtype: float64


In [6]:
arrivees_par_pilote = df['Driver'].value_counts()

print("\n📈 Nombre d'arrivées par pilote :")
print(arrivees_par_pilote.head(10))


📈 Nombre d'arrivées par pilote :
Driver
Norris        7
Verstappen    7
Russell       7
Albon         7
Piastri       7
Antonelli     6
Leclerc       6
Ocon          6
Hamilton      6
Hadjar        6
Name: count, dtype: int64


In [7]:
vainqueurs_par_course = df[df['Position'] == 1][['Round', 'Race', 'Driver', 'Constructor']].sort_values(by='Round')

print("\n🥇 Vainqueur par course :")
print(vainqueurs_par_course)


🥇 Vainqueur par course :
     Round                       Race      Driver Constructor
0        1      Australian Grand Prix      Norris     McLaren
20       2         Chinese Grand Prix     Piastri     McLaren
40       3        Japanese Grand Prix  Verstappen    Red Bull
60       4         Bahrain Grand Prix     Piastri     McLaren
80       5   Saudi Arabian Grand Prix     Piastri     McLaren
100      6           Miami Grand Prix     Piastri     McLaren
120      7  Emilia Romagna Grand Prix  Verstappen    Red Bull


In [10]:
# Total de points par pilote
points_totaux = df.groupby('Driver')['Points'].sum()

# Nombre de victoires
victoires = df[df['Position'] == 1].groupby('Driver').size()

# Position moyenne (plus c’est bas, mieux c’est)
position_moyenne = df.groupby('Driver')['Position'].mean()

# Nombre total de courses disputées
courses_disputees = df.groupby('Driver').size()

# Taux de finition (nb de courses terminées / courses disputées)
# On suppose ici que `df` contient uniquement les courses "terminées"
# Il faudrait une autre variable `df_all` pour toutes les courses (optionnel)
# Pour simplifier, on considère ici que tous les pilotes présents ont terminé toutes leurs courses.

# Score heuristique (vous pouvez ajuster les poids)
score = (
    points_totaux / points_totaux.max() * 0.5 +
    victoires.fillna(0) / victoires.max() * 0.3 +
    (1 / position_moyenne) / (1 / position_moyenne).max() * 0.2
)

# Classement prévisionnel basé sur le score
classement_pronostic = score.sort_values(ascending=False)

print("🔮 Classement pronostiqué pour la prochaine course :")
print(classement_pronostic.head(10))

🔮 Classement pronostiqué pour la prochaine course :
Driver
Piastri       0.968421
Verstappen    0.756970
Norris        0.744697
Albon              NaN
Alonso             NaN
Antonelli          NaN
Bearman            NaN
Bortoleto          NaN
Colapinto          NaN
Doohan             NaN
dtype: float64


In [15]:
# Position moyenne sur la grille
grid_moy = df.groupby('Driver')['Grid'].mean()

# On inverse les moyennes car une plus petite valeur est meilleure
inv_pos_moy = 1 / position_moyenne
inv_grid_moy = 1 / grid_moy

# Score combiné (vous pouvez ajuster les poids)
score = (
    (points_totaux / points_totaux.max()) * 0.4 +
    (victoires.fillna(0) / victoires.max()) * 0.3 +
    (inv_pos_moy / inv_pos_moy.max()) * 0.15 +
    (inv_grid_moy / inv_grid_moy.max()) * 0.15
)

# Classement prévisionnel basé sur le score
classement_pronostic = score.sort_values(ascending=False)

print("🔮 Classement pronostiqué pour la prochaine course :")
print(classement_pronostic.head(10))

🔮 Classement pronostiqué pour la prochaine course :
Driver
Piastri       0.976316
Verstappen    0.738102
Norris        0.675758
Albon              NaN
Alonso             NaN
Antonelli          NaN
Bearman            NaN
Bortoleto          NaN
Colapinto          NaN
Doohan             NaN
dtype: float64
